In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

True


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d nikayz/vr-miniproject-2 --force

Dataset URL: https://www.kaggle.com/datasets/nikayz/vr-miniproject-2
License(s): unknown
 99% 2.81G/2.83G [00:30<00:00, 32.3MB/s]
100% 2.83G/2.83G [00:30<00:00, 101MB/s] 


In [ ]:
!unzip vr-miniproject-2.zip

Streaming output truncated to the last 5000 lines.
  inflating: abo-images-small/images/small/fc/fcc8c5b3.jpg  
  inflating: abo-images-small/images/small/fc/fcc8e63b.jpg  
  inflating: abo-images-small/images/small/fc/fcc90df4.jpg  
  inflating: abo-images-small/images/small/fc/fcc912d0.jpg  
  inflating: abo-images-small/images/small/fc/fcc96812.jpg  
  inflating: abo-images-small/images/small/fc/fcc98592.jpg  
  inflating: abo-images-small/images/small/fc/fcc99a61.jpg  
  inflating: abo-images-small/images/small/fc/fcc9ab67.jpg  
  inflating: abo-images-small/images/small/fc/fcc9b5ec.jpg  
  inflating: abo-images-small/images/small/fc/fcc9e1cc.jpg  
  inflating: abo-images-small/images/small/fc/fcc9fb37.jpg  
  inflating: abo-images-small/images/small/fc/fcca0611.jpg  
  inflating: abo-images-small/images/small/fc/fcca3638.jpg  
  inflating: abo-images-small/images/small/fc/fcca495f.jpg  
  inflating: abo-images-small/images/small/fc/fcca4e4d.jpg  
  inflating: abo-images-small/imag

In [ ]:
# it is used to read image.csv file in abo-images-small data which has image id
import pandas as pd
file_path = pd.read_csv('/content/abo-images-small/images/metadata/images.csv')


In [ ]:
!pip install torch torchvision transformers pandas pillow tqdm bert-score

In [ ]:
import torch
from transformers import ViltProcessor, ViltForQuestionAnswering
from PIL import Image
import pandas as pd
from tqdm import tqdm
import os
from google.colab import files
import zipfile
from bert_score import score

In [ ]:
!kaggle datasets download -d nikayz/qna-1l --force

Dataset URL: https://www.kaggle.com/datasets/nikayz/qna-1l
License(s): unknown
  0% 0.00/0.98M [00:00<?, ?B/s]
100% 0.98M/0.98M [00:00<00:00, 829MB/s]


In [ ]:
!unzip qna-1l.zip

Archive:  qna-1l.zip
  inflating: cleaned_qna.csv         


In [ ]:

# # Load the dataset
# dataset_path = '/content/cleaned_qna_colab.csv'  # Update if your filename is different
# images_dir = '/content/data/images'  # Update this path based on your actual image location

# # Check if file exists
# if not os.path.exists(dataset_path):
#     raise FileNotFoundError(f"Dataset file not found at {dataset_path}. Please check the path.")

df = pd.read_csv("/content/cleaned_qna.csv")

# Show sample data
print("Dataset sample:")
print(df.head())

Dataset sample:
      image_id                         question       answer
0  01dkn0Gyx0L       What is the glass's color?        Clear
1  01dkn0Gyx0L        What is the overall form?  Cylindrical
2  01dkn0Gyx0L            What is the material?        Glass
3  01dkn0Gyx0L        What is the glass finish?       Smooth
4  01dkn0Gyx0L  What is the glass's appearance?  Transparent


In [ ]:
# merging qna with images
final_df = pd.merge(df, file_path, on='image_id', how='left')

In [ ]:
final_df = final_df[["image_id","path","question","answer"]]

In [ ]:
import os
image_folder = '/content/abo-images-small/images/small'
final_df['path'] = final_df['path'].apply(lambda p: os.path.join(image_folder, p))

In [ ]:
final_df.head()

,image_id,path,question,answer
0,01dkn0Gyx0L,/content/abo-images-small/images/small/da/daab...,What is the glass's color?,Clear
1,01dkn0Gyx0L,/content/abo-images-small/images/small/da/daab...,What is the overall form?,Cylindrical
2,01dkn0Gyx0L,/content/abo-images-small/images/small/da/daab...,What is the material?,Glass
3,01dkn0Gyx0L,/content/abo-images-small/images/small/da/daab...,What is the glass finish?,Smooth
4,01dkn0Gyx0L,/content/abo-images-small/images/small/da/daab...,What is the glass's appearance?,Transparent


In [ ]:
from PIL import Image
import os
import torch
from tqdm import tqdm
import pandas as pd
from bert_score import score
from transformers import ViltProcessor, ViltForQuestionAnswering
from sklearn.metrics import precision_score, recall_score, f1_score

# Initialize device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load ViLBERT model and processor
processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa").to(device)

def evaluate_vilbert_with_bertscore(df, sample_size=None):
    """
    Evaluate ViLBERT model with comprehensive metrics including BERTScore

    Args:
        df: DataFrame containing:
            - path: Full image paths
            - qa_questions: List of questions
            - qa_answers: List of answers
        sample_size: Number of samples to evaluate (None for all)

    Returns:
        results_df: DataFrame with detailed results
        metrics: Dictionary of aggregate metrics
    """
    # Initialize results storage
    results = {
        'image_id': [],
        'image_path': [],
        'question': [],
        'true_answer': [],
        'predicted_answer': [],
        'is_correct': [],
        'confidence': []
    }

    if sample_size:
        df = df.sample(min(sample_size, len(df)), random_state=42)

    for _, row in tqdm(df.iterrows(), total=len(df)):
        try:
            img = Image.open(row['path']).convert('RGB')

            questions = str(row["question"]).strip().lower()
            answers = str(row["answer"]).strip().lower()

            q = questions
            true_ans = answers

            inputs = processor(img, q, return_tensors="pt").to(device)

            with torch.no_grad():
                outputs = model(**inputs)
                pred_class = outputs.logits.argmax(-1).item()
                pred_answer = model.config.id2label[pred_class]
                confidence = torch.nn.functional.softmax(outputs.logits, dim=-1).max().item()

            results['image_id'].append(row['image_id'])
            results['image_path'].append(row['path'])
            results['question'].append(q)
            results['true_answer'].append(true_ans)
            results['predicted_answer'].append(pred_answer)
            results['is_correct'].append(pred_answer.lower() == true_ans.lower())
            results['confidence'].append(confidence)

        except Exception as e:
            print(f"Error processing {row['image_id']}: {str(e)}")
            continue

    # Convert to DataFrame
    results_df = pd.DataFrame(results)

    # Calculate standard metrics
    accuracy = results_df['is_correct'].mean()
    avg_confidence = results_df['confidence'].mean()

    # Calculate BERTScore
    if len(results_df) > 0:
        P, R, F1 = score(results_df['predicted_answer'].tolist(),
                        results_df['true_answer'].tolist(),
                        lang='en',
                        verbose=True)

        results_df['bertscore_precision'] = P.numpy()
        results_df['bertscore_recall'] = R.numpy()
        results_df['bertscore_f1'] = F1.numpy()

        bertscore_metrics = {
            'bertscore_precision': P.mean().item(),
            'bertscore_recall': R.mean().item(),
            'bertscore_f1': F1.mean().item()
        }
    else:
        bertscore_metrics = {}

    # Calculate precision/recall/F1 (macro averaged)
    y_true = results_df['is_correct'].astype(int)
    metrics = {
        'accuracy': accuracy,
        'precision': precision_score(y_true, [1]*len(y_true)),  # Always predict correct for macro
        'recall': recall_score(y_true, [1]*len(y_true)),
        'f1': f1_score(y_true, [1]*len(y_true)),
        'avg_confidence': avg_confidence,
        'total_samples': len(results_df),
        'correct_count': results_df['is_correct'].sum(),
        **bertscore_metrics
    }

    return results_df, metrics


Using device: cuda


In [ ]:
pd.set_option('display.max_colwidth', None)
print(df[df['image_id'] == "21jsWeHp5eL"]["path"])

5145    /content/abo-images-small/images/small/5f/5f35a4d0.jpg
5146    /content/abo-images-small/images/small/5f/5f35a4d0.jpg
5147    /content/abo-images-small/images/small/5f/5f35a4d0.jpg
Name: path, dtype: object


In [ ]:
df = final_df[:10000]

In [ ]:
results_table, metrics = evaluate_vilbert_with_bertscore(df)

 52%|█████▏    | 5152/10000 [02:21<01:44, 46.27it/s]

Error processing 21jsWeHp5eL: height and width must be > 0
Error processing 21jsWeHp5eL: height and width must be > 0
Error processing 21jsWeHp5eL: height and width must be > 0


100%|██████████| 10000/10000 [04:33<00:00, 36.59it/s]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/25 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/157 [00:00<?, ?it/s]

done in 2.96 seconds, 3374.16 sentences/sec


In [ ]:

def print_metrics(metrics):
    print("Aggregate Metrics:")
    print(f"- Accuracy:            {metrics['accuracy']:.2%} ({metrics['accuracy']:.4f})")
    print(f"- Precision:           {metrics['precision']:.4f}")
    print(f"- Recall:              {metrics['recall']:.4f}")
    print(f"- F1 Score:            {metrics['f1']:.4f}")
    print(f"- Avg Confidence:      {metrics['avg_confidence']:.4f}")
    print(f"- Total Samples:       {metrics['total_samples']:,}")
    print(f"- Correct Count:       {metrics['correct_count']:,}")
    print(f"- BERTScore Precision: {metrics['bertscore_precision']:.4f}")
    print(f"- BERTScore Recall:    {metrics['bertscore_recall']:.4f}")
    print(f"- BERTScore F1:        {metrics['bertscore_f1']:.4f}")

# Usage:
print_metrics(metrics)

Aggregate Metrics:
- Accuracy:            20.39% (0.2039)
- Precision:           0.2039
- Recall:              1.0000
- F1 Score:            0.3387
- Avg Confidence:      0.4269
- Total Samples:       9,997
- Correct Count:       2,038
- BERTScore Precision: 0.9730
- BERTScore Recall:    0.9638
- BERTScore F1:        0.9677


In [ ]:
print(results_table.head())

      image_id                                         image_path  \
0  01dkn0Gyx0L  /content/abo-images-small/images/small/da/daab...   
1  01dkn0Gyx0L  /content/abo-images-small/images/small/da/daab...   
2  01dkn0Gyx0L  /content/abo-images-small/images/small/da/daab...   
3  01dkn0Gyx0L  /content/abo-images-small/images/small/da/daab...   
4  01dkn0Gyx0L  /content/abo-images-small/images/small/da/daab...   

                          question  true_answer predicted_answer  is_correct  \
0        what is the overall form?  cylindrical            glass       False   
1            what is the material?        glass            glass        True   
2        what is the glass finish?       smooth            clear       False   
3  what is the glass's appearance?  transparent            clear       False   
4         what is the glass style?       simple            clear       False   

   confidence  bertscore_precision  bertscore_recall  bertscore_f1  
0    0.711458             0.903636 

In [ ]:
from PIL import Image
import os
import torch
from tqdm import tqdm
import pandas as pd
from bert_score import score
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from sklearn.metrics import precision_score, recall_score, f1_score

# Initialize device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load BLIP-2 model and processor
processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl")
model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-flan-t5-xl",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
).to(device)

def evaluate_blip2_with_bertscore(df, sample_size=None):
    """
    Evaluate BLIP-2 model with comprehensive metrics including BERTScore

    Args:
        df: DataFrame containing:
            - path: Full image paths
            - qa_questions: List of questions
            - qa_answers: List of answers
        sample_size: Number of samples to evaluate (None for all)

    Returns:
        results_df: DataFrame with detailed results
        metrics: Dictionary of aggregate metrics
    """
    # Initialize results storage
    results = {
        'image_id': [],
        'image_path': [],
        'question': [],
        'true_answer': [],
        'predicted_answer': [],
        'is_correct': [],
        'generation_length': []
    }

    if sample_size:
        df = df.sample(min(sample_size, len(df)), random_state=42)

    for _, row in tqdm(df.iterrows(), total=len(df)):
        try:
            img = Image.open(row['path']).convert('RGB')

            questions = eval(row['qa_questions']) if isinstance(row['qa_questions'], str) else row['qa_questions']
            answers = eval(row['qa_answers']) if isinstance(row['qa_answers'], str) else row['qa_answers']

            for q, true_ans in zip(questions, answers):
                inputs = processor(img, q, return_tensors="pt").to(device, torch.float16)

                with torch.no_grad():
                    generated_ids = model.generate(**inputs)
                    pred_answer = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip().lower()

                results['image_id'].append(row['image_id'])
                results['image_path'].append(row['path'])
                results['question'].append(q)
                results['true_answer'].append(str(true_ans))
                results['predicted_answer'].append(pred_answer)
                results['is_correct'].append(pred_answer.lower() == str(true_ans).lower())
                results['generation_length'].append(len(pred_answer.split()))

        except Exception as e:
            print(f"Error processing {row['image_id']}: {str(e)}")
            continue

    # Convert to DataFrame
    results_df = pd.DataFrame(results)

    # Calculate standard metrics
    accuracy = results_df['is_correct'].mean()
    avg_length = results_df['generation_length'].mean()

    # Calculate BERTScore
    if len(results_df) > 0:
        P, R, F1 = score(results_df['predicted_answer'].tolist(),
                        results_df['true_answer'].tolist(),
                        lang='en',
                        verbose=True)

        results_df['bertscore_precision'] = P.numpy()
        results_df['bertscore_recall'] = R.numpy()
        results_df['bertscore_f1'] = F1.numpy()

        bertscore_metrics = {
            'bertscore_precision': P.mean().item(),
            'bertscore_recall': R.mean().item(),
            'bertscore_f1': F1.mean().item()
        }
    else:
        bertscore_metrics = {}

    # Calculate precision/recall/F1
    y_true = results_df['is_correct'].astype(int)
    metrics = {
        'accuracy': accuracy,
        'precision': precision_score(y_true, [1]*len(y_true)),
        'recall': recall_score(y_true, [1]*len(y_true)),
        'f1': f1_score(y_true, [1]*len(y_true)),
        'avg_generation_length': avg_length,
        'total_samples': len(results_df),
        'correct_count': results_df['is_correct'].sum(),
        **bertscore_metrics
    }

    return results_df, metrics

# Example usage:
# results_table, metrics = evaluate_blip2_with_bertscore(your_dataframe)
# print("Aggregate Metrics:", metrics)
# results_table.to_csv('blip2_evaluation_results.csv', index=False)

Using device: cuda


preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/5.81G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 20.12 MiB is free. Process 39994 has 14.72 GiB memory in use. Of the allocated memory 13.64 GiB is allocated by PyTorch, and 974.05 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
results_table, metrics = evaluate_blip2_with_bertscore(df)
print("Aggregate Metrics:", metrics)
results_table.to_csv('blip2_evaluation_results.csv', index=False)

  0%|          | 3/4074 [00:00<18:36,  3.65it/s]

Error processing 71fu9sfmtNL: shape mismatch: value tensor of shape [81920] cannot be broadcast to indexing result of shape [0]
Error processing 71fuEz4S6DL: shape mismatch: value tensor of shape [81920] cannot be broadcast to indexing result of shape [0]
Error processing 71fuFfPO0RL: shape mismatch: value tensor of shape [81920] cannot be broadcast to indexing result of shape [0]


  0%|          | 5/4074 [00:01<11:55,  5.69it/s]

Error processing 71fuMNqdSsL: shape mismatch: value tensor of shape [81920] cannot be broadcast to indexing result of shape [0]
Error processing 71fuQf5T+8L: shape mismatch: value tensor of shape [81920] cannot be broadcast to indexing result of shape [0]
Error processing 71fuTb7+F-L: shape mismatch: value tensor of shape [81920] cannot be broadcast to indexing result of shape [0]


  0%|          | 9/4074 [00:01<07:40,  8.83it/s]

...

  3%|▎         | 103/4074 [00:07<03:47, 17.42it/s]

Error processing 71fx3uUeMTL: shape mismatch: value tensor of shape [81920] cannot be broadcast to indexing result of shape [0]
Error processing 71fx5ZdKFaL: shape mismatch: value tensor of shape [81920] cannot be broadcast to indexing result of shape [0]
Error processing 71fx83XmdYL: shape mismatch: value tensor of shape [81920] cannot be broadcast to indexing result of shape [0]
Error processing 71fx9f9NvnL: shape mismatch: value tensor of shape [81920] cannot be broadcast to indexing result of shape [0]


  3%|▎         | 106/4074 [00:07<04:39, 14.20it/s]


Error processing 71fxJNgVnZL: shape mismatch: value tensor of shape [81920] cannot be broadcast to indexing result of shape [0]
Error processing 71fxMbxa+hL: shape mismatch: value tensor of shape [81920] cannot be broadcast to indexing result of shape [0]


KeyboardInterrupt: 

In [ ]:
# for BLIP
def print_metrics(metrics):
    print("Aggregate Metrics:")
    print(f"- Accuracy:            {metrics['accuracy']:.2%} ({metrics['accuracy']:.4f})")
    print(f"- Precision:           {metrics['precision']:.4f}")
    print(f"- Recall:              {metrics['recall']:.4f}")
    print(f"- F1 Score:            {metrics['f1']:.4f}")
    print(f"- Avg Confidence:      {metrics['avg_confidence']:.4f}")
    print(f"- Total Samples:       {metrics['total_samples']:,}")
    print(f"- Correct Count:       {metrics['correct_count']:,}")
    print(f"- BERTScore Precision: {metrics['bertscore_precision']:.4f}")
    print(f"- BERTScore Recall:    {metrics['bertscore_recall']:.4f}")
    print(f"- BERTScore F1:        {metrics['bertscore_f1']:.4f}")

# Usage:
print_metrics(metrics)